### **PHISHING VECTOR GENERATOR** 🐟

In [243]:
import pandas as pd
import numpy as np
import json
from copy import deepcopy
import os
import time
import requests
import re
from user_browsing import user_browsing
from xml.etree import ElementTree as ET
from urllib.parse import urlparse
from os import path

In [244]:
class PHISH_FVG:

    def __init__(self, url, fichero = 'html_dump'):

        self.url = url
        self.base, self.path = self.process_url(self.url)

        self.fv = np.array([-1 for i in range(19)])
        self.fichero = fichero

        self.user = user_browsing()
        self.user.set_standard_header(self.base)

        self.html = self.get_source_code()
        self.hyperlinks = self.find_hyperlinks()


    def process_url(self, url):

        parsed = urlparse(url)
        base = parsed.netloc
        return base, base + '/'.join(path.split('/')[:-1])


    def get_source_code(self):

        response = requests.get(self.url) #, headers=self.user.header) #proxies=user.proxies, cookies=user.cookies)

        if response.status_code != 400:
            with open(self.fichero, 'wb') as f:
                f.write(response.content)
                f.close()

        return response.content.decode("utf-8")


    def find_hyperlinks(self):
        return ( re.findall('(?:src\b*=\b*\")([^"]*)(?:\")', self.html) + re.findall('(?:href\b*=\b*\")([^"]*)(?:\")', self.html) )
    
    def url_validator(self, url):
        parsed = urlparse(url)
        return parsed.netloc != ''


    def set_f9(self):

        try:

            forms_found = re.findall("<form[^>]+>", self.html)

            if len(forms_found) > 0:

                for i in range(len(forms_found)):

                    form_found = forms_found[i]
                    action_content = re.findall('(?:action=\")([^"]*)(?:\")', form_found)
                    print(action_content[0])

                    if (len(action_content[0]) <1) or (len(re.findall('javascript:void\(0\)', action_content[0])) > 0):
                        self.fv[8] = 1
                        return

                    elif len(re.findall('^(.*\.php)$', action_content[0])) > 0 and self.base != self.process_url(action_content[0])[0]:
                        self.fv[8] = 1
                        return

                    elif action_content[0][0] != '/' and self.base != self.process_url(action_content[0])[0]:
                        self.fv[8] = 1
                        return
                    
                    else:
                        self.fv[8] = 0
                    
            else:
                self.fv[8] = 0
        
        # No action found
        except:
            self.fv[8] = 0

    
    def set_f10_f11(self):

        try:
            n_hyperlinks_found = len(self.hyperlinks)
            self.fv[9] = n_hyperlinks_found

            if n_hyperlinks_found == 0:
                self.fv[10] = 1

            else:
                self.fv[10] = 0

        except:
            self.fv[9] = 0
            self.fv[10] = 0


    def get_number_foreign_hyperlinks(self):

        n_foreigns = 0

        for h in self.hyperlinks:
            if self.is_foreign(h):
                n_foreigns += 1

        return n_foreigns

        
    def set_f12(self):

        if len(self.hyperlinks) < 1:
            self.fv[11] = 1
            return 0
            
        n_foreigns = self.get_number_foreign_hyperlinks() 
            
        ratio = (n_foreigns / len(self.hyperlinks))

        if ratio < 0.5:
            self.fv[11] = 1

        else:
            self.fv[11] = 0

            # for h in hyperlinks_found:
            #     print("{} {} ".format(h, self.is_foreign(h)))

        print(ratio)


    def set_f13(self):

        if len(self.hyperlinks) < 1:
            self.fv[12] = 1
            return
            
        n_empty = self.get_number_empty_hyperlinks()
            
        ratio = (n_empty / len(self.hyperlinks))

        if ratio > 0.34:
            self.fv[11] = 1

        else:
            self.fv[11] = 0

            # for h in hyperlinks_found:
            #     print("{} {} ".format(h, self.is_foreign(h)))

        print(ratio)


    def set_f14(self):

        if len(self.hyperlinks) < 1:
            self.fv[12] = 1
            return
            
        n_errors = self.get_number_errors()
            
        ratio = (n_errors / len(self.hyperlinks))

        if ratio > 0.3:
            self.fv[11] = 1

        else:
            self.fv[11] = 0

            # for h in hyperlinks_found:
            #     print("{} {} ".format(h, self.is_foreign(h)))

        print(ratio)
    

    
    def get_response_code(self, url):
        return requests.get(url).status_code


    def get_number_empty_hyperlinks(self):

        n_empty = 0

        for h in self.hyperlinks:
            if self.is_empty(h):
                n_empty += 1

        return n_empty


    def is_absolute(self, url):
        return bool(urlparse(url).netloc)

    def is_empty(self, url):
        return url[0] == '#' or bool(re.match('[Jj]ava[Ss]cript::?void\(0\)', url))

    def is_relative_in_local(self, url):

        if self.is_absolute(url):
            return False

        pattern = re.compile("^[/]?[A-z0-9_]")
        return bool (pattern.match(url))
        
    
    def get_number_errors(self):

        n_errors = 0

        for h in self.hyperlinks:

            if not self.is_empty(h) and not self.is_relative_in_local(h):
                code = self.get_response_code(h)

                if code == 404 or code == 403:
                    n_errors += 1

        return n_errors

    def is_foreign(self, url):
        return not self.is_relative_in_local(url) and not self.is_empty(url) and self.base != urlparse(url).netloc

In [245]:
ph_entity = PHISH_FVG('https://ubuvirtual.ubu.es/') # 'https://fdeageadfahgeafeahg.azurewebsites.net/renner/inicio/login.php') # 

In [246]:
# ph_entity.set_f9()

# ph_entity.set_f10_f11()


# print(ph_entity.is_foreign('https://www.ubuvirtual.com/image.jpg'))
# print(ph_entity.is_foreign('www.fdeageadfahgeafeahg.azurewebsites.net/renner/inicio/login.php'))

ph_entity.set_f12()
ph_entity.set_f13()


0.22972972972972974
0.14864864864864866


In [248]:
ph_entity.get_number_errors()

0